In [1]:
import sys
sys.path.insert(0, "../..")
import config as cfg
import gc
import os
from tqdm.notebook import tqdm
from helper import check_path, seed_everything
from collections import defaultdict

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score

In [3]:
import catboost as cb
import catboost.datasets as cbd
import catboost.utils as cbu

In [4]:
import optuna
from optuna.integration import CatBoostPruningCallback

In [5]:
EXPERIMENT_FAMILY_NAME = 'catboost'
EXPERIMENT_NAME = 'deeppavlov_optuna'
N_RANDOM_SEEDS = 1
N_SPLITS = 8

In [6]:
RANDOM_STATE = 77
seed_everything(RANDOM_STATE)

In [7]:
def filter_rare_categories(df: pd.DataFrame) -> pd.DataFrame:
    rare_categories = [12]
    return df.loc[~df[cfg.TARGET].isin(rare_categories)]

In [8]:
train = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'train.pkl'))  #.drop(cfg.TEXT_COL, axis=1)  #  cfg.THEME_COL, cfg.TEXT_COL
test = pd.read_pickle(os.path.join(cfg.PREPROCESSED_DATA_PATH, 'test.pkl'))  #.drop(cfg.TEXT_COL, axis=1) # cfg.TEXT_COL

CLASSES = np.sort(train[cfg.TARGET].unique()).tolist()

# train = filter_rare_categories(train)

In [9]:
emb_names = ['deeppavlov', 'rubert_tiny', 'smaller_LaBSE_15lang']
for emb_name in emb_names:
    train_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, emb_name, 'train.pkl'))
    test_emb = pd.read_pickle(os.path.join(cfg.DATA_PATH, emb_name, 'test.pkl'))

    train = train.join(train_emb)
    test = test.join(test_emb)

del train_emb, test_emb; gc.collect()

0

In [10]:
X_train, y_train = train.drop(cfg.TARGET, axis=1), train[cfg.TARGET]

In [11]:
pred_proba_oof = pd.DataFrame(data=np.zeros(shape=(len(train), len(CLASSES))), index=train.index, columns=CLASSES)
pred_proba_test = pd.DataFrame(data=np.zeros(shape=(len(test), len(CLASSES))), index=test.index, columns=CLASSES)

In [12]:
leak_test = pd.read_pickle(os.path.join(cfg.DATA_PATH, 'test_leak.pkl'))
leak_mask = leak_test.notnull()

In [13]:
train_pool = cb.Pool(
        data=X_train, 
        label=y_train,
        text_features=cfg.TEXT_COLS,
        # cat_features=cfg.CAT_COLS
        )

val_pool = cb.Pool(
        data = test.loc[leak_mask],
        label=leak_test.loc[leak_mask].astype('int'),
        text_features=cfg.TEXT_COLS,
        # cat_features=cfg.CAT_COLS
        )

In [14]:
def objective(trial: optuna.Trial) -> float:

    param = {
        "objective": trial.suggest_categorical("objective", ["MultiClass"]),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.3, log=True),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.5, 10.0, log=False),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
        # "cat_features": CAT_COLS,
        'classes_count': len(CLASSES),
        # 'task_type': "GPU",
        # 'devices': '0:1',
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        # "used_ram_limit": "3gb",
        # "eval_metric": "AUC",
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1, log=True)
    if param['boosting_type'] == 'Ordered':
        param['grow_policy'] = 'SymmetricTree'

    gbm = cb.CatBoostClassifier(**param)

    pruning_callback = CatBoostPruningCallback(trial, param['objective'])
    gbm.fit(
        train_pool,
        eval_set=[val_pool],
        verbose=0,
        early_stopping_rounds=100,
        callbacks=[pruning_callback],
        plot=True
    )

    # evoke pruning manually.
    pruning_callback.check_pruned()

    preds = gbm.predict_proba(val_pool)
    try:
        score = roc_auc_score(leak_test.loc[leak_mask].astype('int'), preds , multi_class='ovo', labels=CLASSES)
    except:
        print('error with auc')
        score = 0
    return score

In [16]:
study = optuna.create_study(
        pruner=optuna.pruners.MedianPruner(n_warmup_steps=7), direction="maximize"
    )
study.optimize(objective, n_trials=100, timeout=28800)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2022-07-24 20:32:22,152] A new study created in memory with name: no-name-071b9d84-6931-4da5-b86b-5a46a09035be
/tmp/ipykernel_54888/1394528551.py:30: ExperimentalWarning: CatBoostPruningCallback is experimental (supported from v3.0.0). The interface can change in the future.
  pruning_callback = CatBoostPruningCallback(trial, param['objective'])


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
best_params = trial.params
if best_params['boosting_type'] == 'Ordered':
        best_params['grow_policy'] = 'SymmetricTree'
best_params['classes_count'] = len(CLASSES)
best_params["eval_metric"] = "AUC"

In [ ]:
cv = StratifiedKFold(n_splits=N_SPLITS, random_state=cfg.RANDOM_STATE, shuffle=True)

test_pool = cb.Pool(
        data=test,
        text_features=cfg.TEXT_COLS,
        # cat_features=cfg.CAT_COLS
        )


metrics = defaultdict(list)
fold = 0
for train_idx, val_idx in tqdm(cv.split(X_train, y_train), total=N_SPLITS):

    train_pool = cb.Pool(
        data=X_train.iloc[train_idx], 
        label=y_train.iloc[train_idx],
        text_features=cfg.TEXT_COLS,
        # cat_features=cfg.CAT_COLS
        )

    val_pool = cb.Pool(
        data=X_train.iloc[val_idx], 
        label=y_train.iloc[val_idx],
        text_features=cfg.TEXT_COLS,
        # cat_features=cfg.CAT_COLS
        )
        
    for random_seed in tqdm(range(N_RANDOM_SEEDS), total=N_RANDOM_SEEDS):

        clf = cb.CatBoostClassifier(
            **best_params,
            silent=True,
            # random_seed=random_seed,
        )

        clf.fit(train_pool, eval_set=val_pool, plot=True)
        
        model_name = f'{EXPERIMENT_NAME}_fold_{fold}_rs_{random_seed}.cbm'
        model_path = os.path.join(cfg.MODELS_PATH, EXPERIMENT_FAMILY_NAME, EXPERIMENT_NAME)
        check_path(model_path)
        clf.save_model(os.path.join(model_path, model_name))
        
        pred_proba_oof_val = clf.predict_proba(val_pool)
        pred_proba_oof_train = clf.predict_proba(train_pool)

        pred_proba_oof.iloc[val_idx, :] += pred_proba_oof_val
        pred_proba_test.iloc[:, :] += clf.predict_proba(test_pool)

        y_train_oof = y_train.iloc[train_idx]
        y_val_oof = y_train.iloc[val_idx]

        train_auc = roc_auc_score(y_train_oof, pred_proba_oof_train , multi_class='ovo', labels=CLASSES)
        val_auc = roc_auc_score(y_val_oof, pred_proba_oof_val , multi_class='ovo', labels=CLASSES)
        metrics['train_auc'].append(train_auc)
        metrics['val_auc'].append(val_auc)
        print('train auc', train_auc, 'val auc', val_auc)

        leak_test_auc_score = roc_auc_score(leak_test.loc[leak_mask].astype('int'), pred_proba_test.loc[leak_mask] / (fold + 1), multi_class='ovo', labels=CLASSES)
        metrics['leak_test_auc_score'].append(leak_test_auc_score)
        print('leak_test_auc_score', leak_test_auc_score)
        
        del clf; gc.collect()
    del train_pool,val_pool; gc.collect() 
        
    fold += 1
pred_proba_oof /= N_RANDOM_SEEDS
pred_proba_test /= (N_SPLITS * N_RANDOM_SEEDS)

  0%|          | 0/5 [00:00<?, ?it/s]

/home/as/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?it/s]

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

train auc 0.9958376579375091 val auc 0.8738261274752328
leak_test_auc_score 0.8464202086291618


...

train auc 0.995900365555246 val auc 0.8609748678927788
leak_test_auc_score 0.8505576796839819


  0%|          | 0/1 [00:00<?, ?it/s]

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Found only 16 unique classes in the data, but have defined 17 classes. Probably something is wrong with data.
Label(s) 12 are not present in the train set. Perhaps, something is wrong with the data.


train auc 0.9935530221800255 val auc 0.8094515579623661
leak_test_auc_score 0.8544459104571387


In [ ]:
oof_auc_score = roc_auc_score(y_train, pred_proba_oof , multi_class='ovo', labels=CLASSES)
print('oof_auc_score', oof_auc_score)

oof_auc_score 0.8094430699029125


In [ ]:
# 0.8230850052944366

In [ ]:
leak_test_auc_score = roc_auc_score(leak_test.loc[leak_mask].astype('int'), pred_proba_test.loc[leak_mask], multi_class='ovo', labels=CLASSES)
print('leak_test_auc_score', leak_test_auc_score)


leak_test_auc_score 0.8544459104571387


In [ ]:
# 0.8285485808217893

In [ ]:
submission = pd.read_csv(cfg.SAMPLE_SUBMIT_PATH).set_index('id')
assert submission.index.equals(pred_proba_test.index)
submission[cfg.TARGET] = pred_proba_test.idxmax(1)

submission_path = os.path.join(cfg.SUBMISSION_PATH, EXPERIMENT_FAMILY_NAME)
check_path(submission_path)
submission.to_csv(os.path.join(submission_path, f'{EXPERIMENT_NAME}.csv'))

pred_proba_oof_path = os.path.join(cfg.OOF_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_oof_path)
pred_proba_oof.to_pickle(os.path.join(pred_proba_oof_path, f'{EXPERIMENT_NAME}.pkl'))

pred_proba_test_path = os.path.join(cfg.TEST_PRED_PATH, EXPERIMENT_FAMILY_NAME)
check_path(pred_proba_test_path)
pred_proba_test.to_pickle(os.path.join(pred_proba_test_path, f'{EXPERIMENT_NAME}.pkl'))